In [ ]:
# Search is as good as its embeddings

import os
import weaviate
from weaviate.classes.init import Auth
from dotenv import load_dotenv
import json
import weaviate.classes.query as wvcq  # Import for filters and metadata

load_dotenv()  # Load environment variables from a .env file
# Best practice: store your credentials in environment variables
weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]
open_ai_api_key = os.environ["OPENAI_API_KEY"]

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
    headers={"X-OpenAI-Api-Key": open_ai_api_key}
)

print(client.is_ready())

True


In [10]:
collection_name = "EcommerceProducts"

In [ ]:
try:

    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=Auth.api_key(weaviate_api_key),
        headers={"X-OpenAI-Api-Key": open_ai_api_key}
    )       
    
    products = client.collections.get(collection_name)

    # Query 1: Pure Semantic Search
    query_text = "what are some of the durable phones?"
    print(f"--- 1. Semantic Search for {query_text} ---")
    response = products.query.near_text(
        query=query_text,
        limit=10,
        return_metadata=wvcq.MetadataQuery(distance=True),
        return_properties=["product_id", "product_title", "product_brand", "product_color"]
    )
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Distance: {item.metadata.distance:.4f}\n")
finally:
    client.close()
    print("Connection closed.")

--- 1. Semantic Search for what are some of the durable phones? ---
{
  "product_color": "",
  "product_brand": "Babes & Gents",
  "product_title": "Eminem 2 iPhone Phone Case (iPhone 6 Plus/6s Plus)",
  "product_id": "B07H7CQB4N"
}
  Distance: 0.5595

{
  "product_brand": "DURARMOR FLEX SLIM CASE",
  "product_color": "We're All Mad Here Cheshire Cat Smile Face",
  "product_id": "B07K5M78CK",
  "product_title": "iPhone 8 Plus Case, DURARMOR FlexArmor iPhone 7 Plus Alice in Wonderland We're All Mad Here Rubber TPU Case ScratchSafe Cover for iPhone 7 Plus / 8 Plus Cheshire Cat"
}
  Distance: 0.5610

{
  "product_id": "B0764LRWWC",
  "product_color": "Metallic Grey",
  "product_title": "Panasonic KX-TGD563M Link2Cell Bluetooth Cordless Phone with Voice Assist and Answering Machine - 3 Handsets (Renewed)",
  "product_brand": "Panasonic"
}
  Distance: 0.5653

Connection closed.


In [ ]:
try:

    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=Auth.api_key(weaviate_api_key),
        headers={"X-OpenAI-Api-Key": open_ai_api_key}
    )
    
    products = client.collections.get(collection_name)

    # Query 2: Hybrid Search (Semantic + Filtering)
    query_text = "Find me some of the best sports shoes for women?" # if we append - but not for girls, it is not able to filter correctly (problem is with the embeddings don't handle negations well)
    print(f"--- 1. Hybrid Search for {query_text} ---")
    response = products.query.near_text(
        query=query_text,
        limit=10,
        filters=wvcq.Filter.all_of([
            wvcq.Filter.by_property("product_color").equal("Grey"),
            wvcq.Filter.by_property("product_brand").equal("Nike")
            ]),
        return_metadata=wvcq.MetadataQuery(distance=True),
        return_properties=["product_id", "product_title", "product_brand", "product_color"]
    )
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Distance: {item.metadata.distance:.4f}\n")
finally:
    client.close()
    print("Connection closed.")

--- 1. Hybrid Search for Find me some of the best sports shoes for women? ---
{
  "product_color": "Multicolour Black White Thunder Grey 2",
  "product_brand": "Nike",
  "product_title": "Nike Men's Air Zoom Pegasus 36 Running Shoes (Black/White-Thunder Grey, 8)",
  "product_id": "B07H8NZ4XR"
}
  Distance: 0.5084

{
  "product_title": "Nike Vapor Ultrafly 3 Keystone Baseball (Toddler/Little Kid/Big Kid) Black/White/Iron Grey 13 Little Kid M",
  "product_brand": "Nike",
  "product_id": "B08LSNGB2K",
  "product_color": "Black/White-iron Grey"
}
  Distance: 0.6064

{
  "product_color": "Habanera Red/Oil Grey/Black",
  "product_brand": "Nike",
  "product_title": "Nike Mens Air Max 200 AQ2568 600 - Size 9.5",
  "product_id": "B07YLDF1ZX"
}
  Distance: 0.6162

{
  "product_color": "Black/White-anthracite-cool Grey",
  "product_brand": "Nike",
  "product_title": "Nike Unisex Downshifter 9 Grade School Running Shoe Black/White-Anthracite-Cool Grey 6Y Regular US Big Kid",
  "product_id": "B07JC

In [60]:
try:

    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=Auth.api_key(weaviate_api_key),
        headers={"X-OpenAI-Api-Key": open_ai_api_key}
    )
    
    products = client.collections.get(collection_name)

    # Query 3: Only Sematic (putting all filters in the query itself)
    query_text = "Find me some of the best Greay Nike sports shoes for women?"
    print(f"--- 1. Hybrid Search for {query_text} ---")
    response = products.query.near_text(
        query=query_text,
        limit=10,
        return_metadata=wvcq.MetadataQuery(distance=True),
        return_properties=["product_id", "product_title", "product_brand", "product_color"]
    )
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Distance: {item.metadata.distance:.4f}\n")
finally:
    client.close()
    print("Connection closed.")

--- 1. Hybrid Search for Find me some of the best Greay Nike sports shoes for women? ---
{
  "product_color": "Multicolour Light Redwood White Pink Quartz 800",
  "product_brand": "Nike",
  "product_title": "Nike Zoom Winflo 6 Lightweight Running Shoe - Women's (8.5, Clay Red/White)",
  "product_id": "B07LDHHDLG"
}
  Distance: 0.4535

{
  "product_color": "Black/Anthracite",
  "product_brand": "Nike",
  "product_title": "Nike Men's Free Rn 2018 Black/Anthracite, Black/Anthracite, 8.5",
  "product_id": "B075ZY29XC"
}
  Distance: 0.4601

{
  "product_color": "Multicolour Black White Thunder Grey 2",
  "product_brand": "Nike",
  "product_title": "Nike Men's Air Zoom Pegasus 36 Running Shoes (Black/White-Thunder Grey, 8)",
  "product_id": "B07H8NZ4XR"
}
  Distance: 0.4625

{
  "product_id": "B077G74NYY",
  "product_color": "Black/White/Anthracite/Racer Pink",
  "product_title": "Nike Girl's Air Zoom Pegasus 35 Running Shoe Black/Racer Pink/White/Anthracite Size 6.5 M US",
  "product_brand"

In [62]:
try:
    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=weaviate_url,
        auth_credentials=Auth.api_key(weaviate_api_key),
        headers={"X-OpenAI-Api-Key": open_ai_api_key}
    )
    
    products = client.collections.get(collection_name)

    # Query 4: Simple Keyword Search
    # query_text = "Find me some of the best hair extensions?"
    search_property = "product_color"
    search_property_value = "Grey"
    print(f"--- Find all products with {search_property} equal to {search_property_value} ---")

    response = products.query.bm25(
        query="nike cute shoes",
        query_properties=["product_title"],
        operator=wvcq.BM25Operator.or_(minimum_match=2),
        limit=10,
        return_metadata=wvcq.MetadataQuery(score=True),
        return_properties=["product_id", "product_title", "product_brand", "product_color"]
    )
    print(response)
    for item in response.objects:
        print(json.dumps(item.properties, indent=2))
        print(f"  Distance: {item.metadata.score}\n")
finally:
    client.close()
    print("Connection closed.")

--- Find all products with product_color equal to Grey ---
QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('05c76b82-163f-4e89-9d16-a8aef9da0b71'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=6.2451019287109375, explain_score=None, is_consistent=None, rerank_score=None), properties={'product_color': '112-white/White', 'product_brand': 'Nike', 'product_title': "Nike Women's NIKE WMNS AIR FORCE 1 '07 BASKETBALL SHOES 9.5 (WHITE/WHITE)", 'product_id': 'B001TXF40I'}, references=None, vector={}, collection='EcommerceProducts'), Object(uuid=_WeaviateUUIDInt('423eae78-f8a2-42d4-b5b0-38106b9e1c8e'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=6.144079208374023, explain_score=None, is_consistent=None, rerank_score=None), properties={'product_title': "Nike Men's Fitness Shoes, Black, Size 10.5", 'product_brand': 'Nike', 'product_id': 'B078NZWTJ2', 'product_color': 'Black/Mtlc